In [1]:
#掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os
import json

# Google Drive 中的資料夾路徑
target_dir = "/content/drive/MyDrive/kaggle_datasets/transactions-fraud-datasets"

ValueError: mount failed

In [ ]:
# 1. 讀取 CSV
df_card = pd.read_csv(os.path.join(target_dir, "cards_data.csv"))
df_tran = pd.read_csv(os.path.join(target_dir, "transactions_data.csv"))
df_user = pd.read_csv(os.path.join(target_dir, "users_data.csv"))

# mcc_codes.json
with open(os.path.join(target_dir, "mcc_codes.json"), "r") as f:
    mcc_data = json.load(f)

# 轉換成兩欄的 DataFrame
df_mcc = pd.DataFrame(list(mcc_data.items()), columns=["mcc_code", "description"])


# train_fraud_labels.json
with open(os.path.join(target_dir, "train_fraud_labels.json"), "r") as f:
    fraud_data = json.load(f)

# fraud_data 如果是 dict，轉換成 DataFrame
if isinstance(fraud_data, dict):
    df_fraud = pd.DataFrame(list(fraud_data.items()), columns=["transaction_id", "is_fraud"])
else:
    df_fraud = pd.DataFrame(fraud_data)

# 確認一下每個資料集大小
print("cards_data.csv:", df_card.shape)
print("transactions_data.csv:", df_tran.shape)
print("users_data.csv:", df_user.shape)
print("mcc_codes.json:", df_mcc.shape)
print("train_fraud_labels.json:", df_fraud.shape)

cards_data.csv: (6146, 13)
transactions_data.csv: (13305915, 12)
users_data.csv: (2000, 14)
mcc_codes.json: (109, 2)
train_fraud_labels.json: (1, 2)


In [ ]:
print(df_card.columns)
print(df_tran.columns)
print(df_user.columns)
print(df_mcc.columns)
print(df_fraud.columns)

Index(['id', 'client_id', 'card_brand', 'card_type', 'card_number', 'expires',
       'cvv', 'has_chip', 'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web'],
      dtype='object')
Index(['id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors'],
      dtype='object')
Index(['id', 'current_age', 'retirement_age', 'birth_year', 'birth_month',
       'gender', 'address', 'latitude', 'longitude', 'per_capita_income',
       'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards'],
      dtype='object')
Index(['mcc_code', 'description'], dtype='object')
Index(['transaction_id', 'is_fraud'], dtype='object')


In [ ]:
cars one heart encoding

SyntaxError: invalid syntax (ipython-input-3595165742.py, line 1)